In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from semopy import Model, Optimizer
import matplotlib.pyplot as plt
from semopy import Model, Optimizer, semplot

In [2]:
def get_adj_close(tickets, start=None, end=None):
    return yf.download(tickets, start=start, end=end)['Adj Close']

In [3]:
tickers = ['BIMBOA.MX', 'KHC', 'NESN.SW', 'PEP', 'KOF', 'MKC', 'HERDEZ.MX', 'BN.PA', 'ULVR.L', 'HLF', 'CAG', 'MDLZ']
closes = get_adj_close(tickers, start='2018-01-01', end='2024-01-01')
rend = closes.pct_change()
rend = rend.dropna()
rend.head()

[*********************100%%**********************]  12 of 12 completed


Ticker,BIMBOA.MX,BN.PA,CAG,HERDEZ.MX,HLF,KHC,KOF,MDLZ,MKC,NESN.SW,PEP,ULVR.L
Date,,,,,,,,,,,,
2018-01-04,0.006221,0.014835,0.016340,-0.006970,0.009734,0.011687,0.003143,0.002349,0.006307,-0.000959,0.004926,0.003080
2018-01-05,-0.002977,0.018308,0.003163,0.023909,0.011079,-0.000898,0.008829,0.012887,0.009793,0.005282,0.002873,0.005281
2018-01-08,-0.000689,-0.006690,-0.014188,0.000000,-0.007400,0.008222,0.010022,-0.005089,0.001067,-0.001194,-0.005730,-0.000244
2018-01-09,0.002758,-0.001403,-0.003465,-0.009426,0.007455,0.001784,-0.009224,-0.000465,-0.000969,0.009086,0.000763,-0.001833
2018-01-10,-0.012835,-0.029928,-0.014977,-0.024221,-0.019496,-0.009285,-0.016787,-0.008374,-0.014934,-0.014218,-0.005081,-0.019221


In [4]:
model_desc = f"""
# Latent Variables
Mood =~ {tickers[0]} + {tickers[1]} + {tickers[2]} + {tickers[3]} + {tickers[4]} + {tickers[5]} + {tickers[6]} + {tickers[7]} + {tickers[8]} + {tickers[9]} + {tickers[10]} + {tickers[11]}
"""

In [5]:
mod = Model(model_desc)
res_opt = mod.fit(rend)
estimates = mod.inspect()

# Imprimir los resultados del ajuste del modelo
print(estimates)

         lval  op       rval   Estimate    Std. Err    z-value   p-value
0   BIMBOA.MX   ~       Mood   1.000000           -          -         -
1         KHC   ~       Mood   0.105632  146.475087   0.000721  0.999425
2     NESN.SW   ~       Mood   0.087062  442.198526   0.000197  0.999843
3         PEP   ~       Mood   0.119906   265.37017   0.000452  0.999639
4         KOF   ~       Mood   0.219131    77.33713   0.002833  0.997739
5         MKC   ~       Mood   0.107729  235.670925   0.000457  0.999635
6   HERDEZ.MX   ~       Mood   0.200873   72.459434   0.002772  0.997788
7       BN.PA   ~       Mood   0.091190  339.278964   0.000269  0.999786
8      ULVR.L   ~       Mood   0.095511  350.709497   0.000272  0.999783
9         HLF   ~       Mood   0.136453   25.825325   0.005284  0.995784
10        CAG   ~       Mood   0.080380  221.467127   0.000363   0.99971
11       MDLZ   ~       Mood   0.141270  218.034635   0.000648  0.999483
12       Mood  ~~       Mood   0.000022    0.009112

In [6]:
# Paso 5: Visualizar el Modelo SEM con un Diagrama de Trayectoria
print(res_opt)

Name of objective: MLW
Optimization method: SLSQP
Optimization successful.
Optimization terminated successfully
Objective value: 90.347
Number of iterations: 25
Params: 0.106 0.087 0.120 0.219 0.108 0.201 0.091 0.096 0.136 0.080 0.141 2.185 8.480 3.607 0.298 0.001 1.559 0.001 3.465 4.075 0.000 14.420 5.167 9.060


In [7]:
mood_factors = res_opt.x[0:(len(tickers)-1)].tolist()
mood_factors.insert(0, 1)
mood_factors

[1,
 0.10563210893786426,
 0.08706157447359421,
 0.11990594864721889,
 0.21913089582630121,
 0.10772942948350947,
 0.2008729960112427,
 0.09118994242852611,
 0.09551086192822093,
 0.13645298442431267,
 0.08037984403624006,
 0.14126960642545952]

In [8]:
def calculate_latent_values(df, factors, var):
    resultado = pd.DataFrame()
    for i in range(len(factors)):
        resultado[var[i]] =  df.iloc[:, i] * factors[i]
    resultado['Latent Value'] = resultado.sum(axis=1)
    resultado = resultado['Latent Value']
    return resultado

In [9]:
mood = calculate_latent_values(rend[tickers], mood_factors, tickers)
mood

Date
2018-01-04    0.012551
2018-01-05    0.011286
2018-01-08   -0.001803
2018-01-09    0.000180
2018-01-10   -0.035428
                ...   
2023-12-22    0.009511
2023-12-26    0.000217
2023-12-27   -0.003364
2023-12-28   -0.004911
2023-12-29   -0.013132
Name: Latent Value, Length: 1550, dtype: float64

In [10]:
g = semplot(mod, 'pd.png')

## Gráfica

<img src="pd.png">